In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 4627, done.
remote: Counting objects: 100% (1573/1573), done.
remote: Compressing objects: 100% (598/598), done.
remote: Total 4627 (delta 1113), reused 1386 (delta 954), pack-reused 3054
Receiving objects: 100% (4627/4627), 2.42 GiB | 23.07 MiB/s, done.
Resolving deltas: 100% (3595/3595), done.
Updating files: 100% (449/449), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 18.2 MB/s eta 0:00:00


In [3]:
! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:1109:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorizationImpressions_Cython_Epoch.c: In functio

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
import scipy.sparse as sps
from concurrent.futures import ThreadPoolExecutor

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv
/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv
/kaggle/input/training-tuned-models/slim_bpr_xgboost_train_validation.zip
/kaggle/input/training-tuned-models/__results__.html
/kaggle/input/training-tuned-models/slim_bpr_all.zip
/kaggle/input/training-tuned-models/__notebook__.ipynb
/kaggle/input/training-tuned-models/ials_xgboost_train_validation.zip
/kaggle/input/training-tuned-models/ials_train_validation.zip
/kaggle/input/training-tuned-models/__output__.json
/kaggle/input/training-tuned-models/slim_bpr_train_validation.zip
/kaggle/input/training-tuned-models/ials_all.zip
/kaggle/input/training-tuned-models/custom.css
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/run_all_algorithms.py
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/cython_example_FunkSVD.py
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/LICENSE
/kaggle/input/training-tuned-models/R

In [5]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [6]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender, SLIMElasticNetRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender, PureSVDItemRecommender, ScaledPureSVDRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.SLIM.SLIM_BPR_Python import SLIM_BPR_Python
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender
from Recommenders.BaseRecommender import BaseRecommender

from Recommenders.Hybrid.LinearCombinationRecommender import LinearCombinationRecommender
from Data_manager.CrossValidationSplitter import CrossValidationSplitter

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

In [7]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [8]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [9]:
config = {
    'tune_parameters': True,
    'database_path': '/kaggle/working/tuning_normalized_lr.db',
    'copy_prev_tuning_db': True,
    'copy_prev_best_params': True,
    'save_github': True
}

In [10]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [11]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [12]:
URM_train, URM_test_complete = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_validation, URM_test = split_train_in_two_percentage_global_sample(URM_test_complete, train_percentage = 0.8)

In [13]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_item.json', 'r') as params_file:
    params_item_knn = json.load(params_file)
params_item_knn

{'topK': 14,
 'shrink': 25,
 'similarity': 'tversky',
 'tversky_alpha': 0.18163095712123836,
 'tversky_beta': 1.8529812241698465}

In [14]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_RP3.json', 'r') as params_file:
    params_rp3 = json.load(params_file)
params_rp3

{'alpha': 0.3408516511081212, 'beta': 0.1447198697205665, 'topK': 32}

In [15]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_user.json', 'r') as params_file:
    params_user_knn = json.load(params_file)
params_user_knn

{'topK': 498,
 'shrink': 0,
 'similarity': 'asymmetric',
 'asymmetric_alpha': 0.39792525383867455}

In [16]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_alpha.json', 'r') as params_file:
    params_p3alpha = json.load(params_file)
params_p3alpha

{'normalize_similarity': 'True',
 'implicit': 'True',
 'alpha': 0.29354071531607107,
 'topK': 49}

In [17]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_hybrid_merging.json', 'r') as params_file:
    params_merging = json.load(params_file)
params_merging

{'alpha': 0.3375591188839621, 'topK': 86}

In [18]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/pure_svd_params.json', 'r') as params_file:
    params_pure_svd = json.load(params_file)
params_pure_svd

{'factors': 30}

In [19]:
# with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_puresvd_item.json', 'r') as params_file:
#     params_pure_svd_item = json.load(params_file)
params_pure_svd_item = {"topK": 43, "num_factors": 171}

In [20]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_puresvd_scaled.json', 'r') as params_file:
    params_scaled = json.load(params_file)
params_scaled

{'num_factors': 179,
 'scaling_items': 5.3298425368534826e-05,
 'scaling_users': 0.2791990262721904}

In [21]:
shutil.copyfile(
    '/kaggle/input/training-tuned-models/ials_train_validation.zip', 
    '/kaggle/working/ials_train_validation.zip', 
)
shutil.copyfile(
    '/kaggle/input/training-tuned-models/ials_all.zip', 
    '/kaggle/working/ials_all.zip', 
)

'/kaggle/working/ials_all.zip'

In [22]:
from numpy import linalg as LA


class NormalizedScoresLinearCombinationRecommender(BaseRecommender):
    def __init__(self, URM_train, recommenders: list[BaseRecommender], weights: list[float], normalize=None):
        super(NormalizedScoresLinearCombinationRecommender, self).__init__(URM_train)
        self.recommenders = recommenders
        weights_sum = sum(weights)
        self.weights = [w / weights_sum for w in weights]
        if normalize is not None and normalize not in ['l1', 'l2', 'linf', 'lminusinf']:
            raise ValueError('Normalization term not recognized')
        self.normalize = normalize
        
    def _compute_item_score(self, user_id_array, items_to_compute=None):
        scores = np.zeros((len(user_id_array), self.n_items))
        for recommender, w in zip(self.recommenders, self.weights):
            recommender_scores = recommender._compute_item_score(user_id_array, items_to_compute)
            scores += self._normalize_scores(recommender_scores) * w
        return scores
    
    def _normalize_scores(self, scores):       
        if self.normalize is None:
            return scores
        elif self.normalize == 'l1':
            return scores / (LA.norm(scores, 1, axis=1, keepdims=True) + 1e-6)
        elif self.normalize == 'l2':
            return scores / (LA.norm(scores, 2, axis=1, keepdims=True) + 1e-6)
        elif self.normalize == 'linf':
            return scores / (LA.norm(scores, np.inf, axis=1, keepdims=True) + 1e-6)
        else:
            return scores / (LA.norm(scores, -np.inf, axis=1, keepdims=True) + 1e-6)

In [23]:
folds = []

for i in tqdm.auto.tqdm(range(10)):
    fold_dict = {}
    fold_dict['URM_train'] = sps.load_npz(f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}/URM_train.npz')
    fold_dict['URM_valid'] = sps.load_npz(f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}/URM_valid.npz')
    fold_dict['Evaluator'] = EvaluatorHoldout(fold_dict['URM_valid'], cutoff_list=[10])
    
    recommender_item = ItemKNNCFRecommender(fold_dict['URM_train'])
    recommender_item.load_model(
        f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}', 'item_knn.zip')
    fold_dict['ItemKNN'] = recommender_item

    recommender_user = UserKNNCFRecommender(fold_dict['URM_train'])
    recommender_user.load_model(
        f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}', 'user_knn.zip')
    fold_dict['UserKNN'] = recommender_user

    recommender_rp3 = RP3betaRecommender(fold_dict['URM_train'])
    recommender_rp3.load_model(
        f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}', 'rp3.zip')
    fold_dict['RP3'] = recommender_rp3

    recommender_p3a = P3alphaRecommender(fold_dict['URM_train'])
    recommender_p3a.load_model(
        f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}', 'p3a.zip')
    fold_dict['P3alpha'] = recommender_p3a

    recommender_pure_svd = PureSVDRecommender(fold_dict['URM_train'])
    recommender_pure_svd.load_model(
        f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}', 'pure_svd.zip')
    fold_dict['PureSVD'] = recommender_pure_svd

    recommender_pure_svd_item = PureSVDItemRecommender(fold_dict['URM_train'])
    recommender_pure_svd_item.load_model(
        f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}', 'item_pure_svd.zip')
    fold_dict['PureSVDItem'] = recommender_pure_svd_item

    recommender_scaled_svd = ScaledPureSVDRecommender(fold_dict['URM_train'])
    recommender_scaled_svd.load_model(
        f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}', 'pure_svd_scaled.zip')
    fold_dict['ScaledPureSVD'] = recommender_scaled_svd

    recommender_nmf = NMFRecommender(fold_dict['URM_train'])
    recommender_nmf.load_model(
        f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}', 'nmf.zip')
    fold_dict['NMF'] = recommender_nmf

    recommender_slim_bpr = SLIM_BPR_Python(fold_dict['URM_train'])
    recommender_slim_bpr.load_model(
        f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}', 'slim_bpr.zip')
    fold_dict['SlimBPR'] = recommender_slim_bpr

    recommender_slim = SLIMElasticNetRecommender(fold_dict['URM_train'])
    recommender_slim.load_model(
        f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}', 'slim_elastic.zip')
    fold_dict['SlimElastic'] = recommender_slim
    
    recommender_ease_r = EASE_R_Recommender(fold_dict['URM_train'])
    recommender_ease_r.load_model(
        f'/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_{i}', 'ease_r.zip')
    fold_dict['EaseR'] = recommender_ease_r   
    
    folds.append(fold_dict)

  0%|          | 0/10 [00:00<?, ?it/s]

EvaluatorHoldout: Ignoring 4017 (30.8%) Users that have less than 1 test interactions
ItemKNNCFRecommender: URM Detected 490 ( 3.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 176 ( 0.8%) items with no interactions.
ItemKNNCFRecommender: Loading model from file '/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_0item_knn.zip'
ItemKNNCFRecommender: Loading complete
UserKNNCFRecommender: URM Detected 490 ( 3.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 176 ( 0.8%) items with no interactions.
UserKNNCFRecommender: Loading model from file '/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_0user_knn.zip'
UserKNNCFRecommender: Loading complete
RP3betaRecommender: URM Detected 490 ( 3.8%) users with no interactions.
RP3betaRecommender: URM Detected 176 ( 0.8%) items with no interactions.
RP3betaRecommender: Loading model from file '/kaggle/working/Recsys_Challenge_2023/models/10Folds/fold_0rp3.zip'
RP3betaRecommender: Loading complete


In [24]:
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

def objective(trial):
    weights = [
        trial.suggest_float('SlimElastic', 1e-2, 1, log=True),
        trial.suggest_float('ItemKNN', 1e-3, 1, log=True),
        trial.suggest_float('RP3', 1e-3, 1, log=True),
        trial.suggest_float('P3alpha', 1e-3, 1, log=True),
        # trial.suggest_float('IALS', 1e-4, 1, log=True),
        trial.suggest_float('EaseR', 1e-4, 1, log=True),
        trial.suggest_float('UserKNN', 1e-4, 1, log=True),
        trial.suggest_float('PureSVD', 1e-5, 0.2, log=True),
        trial.suggest_float('PureSVDItem', 1e-5, 0.2, log=True),
        trial.suggest_float('ScaledPureSVD', 1e-5, 0.2, log=True),
        trial.suggest_float('NMF', 1e-5, 0.2, log=True),
        trial.suggest_float('SlimBPR', 1e-6, 0.2, log=True),
    ]
    
    normalize = trial.suggest_categorical('normalize', ['l1', 'l2', 'linf'])
    
    results = []
    for fold_dict in folds:
        recommenders = [
            fold_dict['SlimElastic'], fold_dict['ItemKNN'], fold_dict['RP3'], fold_dict['P3alpha'],
            fold_dict['EaseR'], fold_dict['UserKNN'], fold_dict['PureSVD'], fold_dict['PureSVDItem'],
            fold_dict['ScaledPureSVD'], fold_dict['NMF'], fold_dict['SlimBPR']
        ]
    
        normalized_linear_comb = NormalizedScoresLinearCombinationRecommender(
            fold_dict['URM_train'], recommenders, weights, normalize=normalize)
        result_df, _ = fold_dict['Evaluator'].evaluateRecommender(normalized_linear_comb)
        results.append(result_df.loc[10, 'MAP'])

    return np.mean(results)

EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions


In [25]:
try:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/tuning_normalized_lr.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass

In [26]:
if config['tune_parameters']:
    study = optuna.create_study(direction='maximize', study_name='hyperparameters_tuning_forcing_norm_linear_11models_10_folds', 
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective, n_trials=50)

[I 2023-12-13 09:52:59,116] Using an existing study with name 'hyperparameters_tuning_forcing_norm_linear_11models_10_folds' instead of creating a new one.


Recommender_Base_Class: URM Detected 490 ( 3.8%) users with no interactions.
Recommender_Base_Class: URM Detected 176 ( 0.8%) items with no interactions.
EvaluatorHoldout: Processed 9007 (100.0%) in 56.96 sec. Users per second: 158
Recommender_Base_Class: URM Detected 493 ( 3.8%) users with no interactions.
Recommender_Base_Class: URM Detected 171 ( 0.8%) items with no interactions.
EvaluatorHoldout: Processed 8987 (100.0%) in 56.54 sec. Users per second: 159
Recommender_Base_Class: URM Detected 604 ( 4.6%) users with no interactions.
Recommender_Base_Class: URM Detected 244 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 9033 (100.0%) in 57.84 sec. Users per second: 156
Recommender_Base_Class: URM Detected 753 ( 5.8%) users with no interactions.
Recommender_Base_Class: URM Detected 363 ( 1.6%) items with no interactions.
EvaluatorHoldout: Processed 9037 (100.0%) in 54.15 sec. Users per second: 167
Recommender_Base_Class: URM Detected 923 ( 7.1%) users with no interacti

[I 2023-12-13 10:01:57,821] Trial 150 finished with value: 0.020419307224394587 and parameters: {'SlimElastic': 0.7654493414550284, 'ItemKNN': 0.12160499926551901, 'RP3': 0.2621721762246877, 'P3alpha': 0.008536127797815018, 'EaseR': 0.035803373882905505, 'UserKNN': 0.018815350781831892, 'PureSVD': 0.007714730955191456, 'PureSVDItem': 5.277868606839039e-05, 'ScaledPureSVD': 8.323975272557137e-05, 'NMF': 0.0016055154282735731, 'SlimBPR': 3.97119056548499e-06, 'normalize': 'l2'}. Best is trial 103 with value: 0.02043634978482866.


Recommender_Base_Class: URM Detected 490 ( 3.8%) users with no interactions.
Recommender_Base_Class: URM Detected 176 ( 0.8%) items with no interactions.
EvaluatorHoldout: Processed 9007 (100.0%) in 57.62 sec. Users per second: 156
Recommender_Base_Class: URM Detected 493 ( 3.8%) users with no interactions.
Recommender_Base_Class: URM Detected 171 ( 0.8%) items with no interactions.
EvaluatorHoldout: Processed 8987 (100.0%) in 58.13 sec. Users per second: 155
Recommender_Base_Class: URM Detected 604 ( 4.6%) users with no interactions.
Recommender_Base_Class: URM Detected 244 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 9033 (100.0%) in 56.96 sec. Users per second: 159
Recommender_Base_Class: URM Detected 753 ( 5.8%) users with no interactions.
Recommender_Base_Class: URM Detected 363 ( 1.6%) items with no interactions.
EvaluatorHoldout: Processed 9037 (100.0%) in 55.25 sec. Users per second: 164
Recommender_Base_Class: URM Detected 923 ( 7.1%) users with no interacti

[I 2023-12-13 10:10:53,145] Trial 151 finished with value: 0.020414422330572446 and parameters: {'SlimElastic': 0.5457980206507874, 'ItemKNN': 0.100937308442707, 'RP3': 0.22061837855475006, 'P3alpha': 0.004714545598056013, 'EaseR': 0.030839569312357623, 'UserKNN': 0.010312691340709114, 'PureSVD': 0.0031941241535833084, 'PureSVDItem': 6.862673740652239e-05, 'ScaledPureSVD': 0.00023371742218140606, 'NMF': 0.0031124633657731524, 'SlimBPR': 2.181707939752055e-06, 'normalize': 'l2'}. Best is trial 103 with value: 0.02043634978482866.


...

[I 2023-12-13 16:44:44,328] Trial 197 finished with value: 0.02037807424898199 and parameters: {'SlimElastic': 0.6944571248490601, 'ItemKNN': 0.08347411544465731, 'RP3': 0.16789392415217425, 'P3alpha': 0.0021660214226774414, 'EaseR': 0.10780394842132303, 'UserKNN': 0.00737834221125431, 'PureSVD': 0.004571922297853587, 'PureSVDItem': 4.713941130755577e-05, 'ScaledPureSVD': 0.0005335412776463372, 'NMF': 0.003288039855256378, 'SlimBPR': 1.6703848782199058e-06, 'normalize': 'l2'}. Best is trial 103 with value: 0.02043634978482866.


Recommender_Base_Class: URM Detected 490 ( 3.8%) users with no interactions.
Recommender_Base_Class: URM Detected 176 ( 0.8%) items with no interactions.
EvaluatorHoldout: Processed 9007 (100.0%) in 55.02 sec. Users per second: 164
Recommender_Base_Class: URM Detected 493 ( 3.8%) users with no interactions.
Recommender_Base_Class: URM Detected 171 ( 0.8%) items with no interactions.
EvaluatorHoldout: Processed 8987 (100.0%) in 53.40 sec. Users per second: 168
Recommender_Base_Class: URM Detected 604 ( 4.6%) users with no interactions.
Recommender_Base_Class: URM Detected 244 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 9033 (100.0%) in 52.64 sec. Users per second: 172
Recommender_Base_Class: URM Detected 753 ( 5.8%) users with no interactions.
Recommender_Base_Class: URM Detected 363 ( 1.6%) items with no interactions.
EvaluatorHoldout: Processed 9037 (100.0%) in 55.44 sec. Users per second: 163
Recommender_Base_Class: URM Detected 923 ( 7.1%) users with no interacti

[I 2023-12-13 16:53:04,963] Trial 198 finished with value: 0.020189476810085722 and parameters: {'SlimElastic': 0.1682010616314369, 'ItemKNN': 0.1826056489865899, 'RP3': 0.08355867712453546, 'P3alpha': 0.002771208262733651, 'EaseR': 0.03966905084022585, 'UserKNN': 0.013235904031020465, 'PureSVD': 0.0031568591928202707, 'PureSVDItem': 6.45687551891148e-05, 'ScaledPureSVD': 0.0007334172362569879, 'NMF': 0.002185336257180063, 'SlimBPR': 4.602469211478076e-06, 'normalize': 'l2'}. Best is trial 103 with value: 0.02043634978482866.


Recommender_Base_Class: URM Detected 490 ( 3.8%) users with no interactions.
Recommender_Base_Class: URM Detected 176 ( 0.8%) items with no interactions.
EvaluatorHoldout: Processed 9007 (100.0%) in 54.15 sec. Users per second: 166
Recommender_Base_Class: URM Detected 493 ( 3.8%) users with no interactions.
Recommender_Base_Class: URM Detected 171 ( 0.8%) items with no interactions.
EvaluatorHoldout: Processed 8987 (100.0%) in 53.30 sec. Users per second: 169
Recommender_Base_Class: URM Detected 604 ( 4.6%) users with no interactions.
Recommender_Base_Class: URM Detected 244 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 9033 (100.0%) in 52.67 sec. Users per second: 172
Recommender_Base_Class: URM Detected 753 ( 5.8%) users with no interactions.
Recommender_Base_Class: URM Detected 363 ( 1.6%) items with no interactions.
EvaluatorHoldout: Processed 9037 (100.0%) in 50.91 sec. Users per second: 178
Recommender_Base_Class: URM Detected 923 ( 7.1%) users with no interacti

[I 2023-12-13 17:01:22,676] Trial 199 finished with value: 0.020373082321769032 and parameters: {'SlimElastic': 0.9082200632951919, 'ItemKNN': 0.1292242547446175, 'RP3': 0.10018558456890167, 'P3alpha': 0.0016542023914599398, 'EaseR': 0.023867763237428886, 'UserKNN': 0.005466940556729004, 'PureSVD': 0.00573669601285782, 'PureSVDItem': 2.74627808055208e-05, 'ScaledPureSVD': 0.0004255529584300366, 'NMF': 0.0013131783018210852, 'SlimBPR': 1.2904911356395259e-06, 'normalize': 'l2'}. Best is trial 103 with value: 0.02043634978482866.


In [27]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [28]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()

In [29]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [30]:
if config['tune_parameters']:
    with open('/kaggle/working/best_params_hybrid_norm_lr_comb_11models.json', 'w') as params_file:
        json.dump(study.best_params, params_file)
        
    if config['save_github']: 
        upload_file(
            '/kaggle/working/best_params_hybrid_norm_lr_comb_11models.json', 
            'tuning_results/best_params_hybrid_norm_lr_comb_11models.json', 
            'Normalized linear combination of 11 models tuning results (from kaggle notebook)'
        )
    
elif config['copy_prev_best_params']:
    shutil.copyfile(
        '/kaggle/input/normalized-linear-combination/best_params_hybrid_norm_lr_comb_11models.json', 
        '/kaggle/working/best_params_hybrid_norm_lr_comb_11models.json'
    )

In [31]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'],
        'tuning_results/tuning_normalized_lr.db', 
        'Linear combination tuning db updated results (from kaggle notebook)'
    )

In [32]:
with open('/kaggle/working/best_params_hybrid_norm_lr_comb_11models.json', 'r') as params_file:
    params = json.load(params_file)
params

{'SlimElastic': 0.7030410917925037,
 'ItemKNN': 0.14418783497150656,
 'RP3': 0.16045150245715412,
 'P3alpha': 0.02079294458867632,
 'EaseR': 0.02876081772920291,
 'UserKNN': 0.005795699832970975,
 'PureSVD': 0.003445251592527658,
 'PureSVDItem': 6.0926156180209626e-05,
 'ScaledPureSVD': 2.3220740825429865e-05,
 'NMF': 0.0013427069325770609,
 'SlimBPR': 4.357612190590355e-06,
 'normalize': 'l2'}

In [33]:
recommender_item = ItemKNNCFRecommender(URM_all)
recommender_item.fit(**params_item_knn)

recommender_user = UserKNNCFRecommender(URM_all)
recommender_user.fit(**params_user_knn)

recommender_rp3 = RP3betaRecommender(URM_all)
recommender_rp3.fit(**params_rp3)

recommender_p3a = P3alphaRecommender(URM_all)
recommender_p3a.fit(**params_p3alpha)

recommender_pure_svd = PureSVDRecommender(URM_all)
recommender_pure_svd.fit(num_factors=params_pure_svd['factors'])

recommender_pure_svd_item = PureSVDItemRecommender(URM_all)
recommender_pure_svd_item.fit(**params_pure_svd_item)

recommender_scaled_svd = ScaledPureSVDRecommender(URM_all)
recommender_scaled_svd.fit(**params_scaled) 

recommender_nmf = NMFRecommender(URM_all)
recommender_nmf.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'nmf_all.zip')

recommender_slim_bpr = SLIM_BPR_Python(URM_all)
recommender_slim_bpr.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'slim_bpr_all.zip')

recommender_slim = SLIMElasticNetRecommender(URM_all)
recommender_slim.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'slim_elastic_all.zip')

recommender_ease_r = EASE_R_Recommender(URM_all)
recommender_ease_r.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'ease_r_all.zip')

ItemKNNCFRecommender: URM Detected 386 ( 3.0%) users with no interactions.
ItemKNNCFRecommender: URM Detected 125 ( 0.6%) items with no interactions.
Similarity column 22347 (100.0%), 4182.52 column/sec. Elapsed time 5.34 sec
UserKNNCFRecommender: URM Detected 386 ( 3.0%) users with no interactions.
UserKNNCFRecommender: URM Detected 125 ( 0.6%) items with no interactions.
Similarity column 13024 (100.0%), 4009.57 column/sec. Elapsed time 3.25 sec
RP3betaRecommender: URM Detected 386 ( 3.0%) users with no interactions.
RP3betaRecommender: URM Detected 125 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 2336.09 column/sec. Elapsed time 9.57 sec
P3alphaRecommender: URM Detected 386 ( 3.0%) users with no interactions.
P3alphaRecommender: URM Detected 125 ( 0.6%) items with no interactions.
P3alphaRecommender: Similarity column 22347 (100.0%), 2687.00 column/sec. Elapsed time 8.32 sec
PureSVDRecommender: URM Detected 386 ( 3.0%) users with no inter

In [34]:
weights = [
    params['SlimElastic'],
    params['ItemKNN'],
    params['RP3'],
    params['P3alpha'],
    params['EaseR'],
    params['UserKNN'],
    params['PureSVD'],
    params['PureSVDItem'],
    params['ScaledPureSVD'],
    params['NMF'],
    params['SlimBPR']
]

recommenders = [
    recommender_slim, recommender_item, recommender_rp3, recommender_p3a, recommender_ease_r,
    recommender_user, recommender_pure_svd, recommender_pure_svd_item, recommender_scaled_svd, 
    recommender_nmf, recommender_slim_bpr
]

In [35]:
normalized_linear_comb = NormalizedScoresLinearCombinationRecommender(
    URM_all,
    recommenders=recommenders,
    weights=weights,
    normalize=params['normalize']
)
recommendations = normalized_linear_comb.recommend(user_ids_test['user_id'].unique() - 1, cutoff=10, return_scores=False)
recommendations = [' '.join([str(item + 1) for item in r]).strip() for r in recommendations]

submission_df = pd.DataFrame({'user_id': user_ids_test['user_id'].unique(), 'item_list': recommendations})
display(submission_df.head())

submission_df.to_csv('/kaggle/working/submission_norm_linear_comb_11models.csv', index=False)

Recommender_Base_Class: URM Detected 386 ( 3.0%) users with no interactions.
Recommender_Base_Class: URM Detected 125 ( 0.6%) items with no interactions.


,user_id,item_list
0,1,36 101 123 506 403 515 694 1546 52 1422
1,2,1095 47 12 1522 50 28 656 11 102 196
2,3,59 857 2172 4252 1281 259 648 4 536 239
3,4,249 28 50 139 314 171 146 7 254 136
4,5,1570 77 170 131 95 5138 1511 471 116 135
